# Loading Data

Reference - https://python.langchain.com/docs/tutorials/rag/

In [ ]:
# !git clone https://github.com/CalvQ/RAGnaRec.git
# import os

fatal: destination path 'RAGnaRec' already exists and is not an empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma
!pip install -U langchain langchain-openai
!pip install sentence-transformers langchain chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
 

In [ ]:
!pip install seaborn

In [ ]:

# Import packages
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import random

import getpass
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DataFrameLoader
from transformers import pipeline

In [ ]:
!pip install pyarrow fastparquet huggingface_hub matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.4 MB/s eta 0:00:00


In [ ]:
splits = {'train': 'yelp_review_full/train-00000-of-00001.parquet', 'test': 'yelp_review_full/test-00000-of-00001.parquet'}
df_train = pd.read_parquet("hf://datasets/Yelp/yelp_review_full/" + splits["train"])
df_test = pd.read_parquet("hf://datasets/Yelp/yelp_review_full/" + splits["test"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df = pd.concat([df_train, df_test], ignore_index=True)
index_list = range(len(df))
random.seed(10701)
indices = random.sample(index_list, 1000)
df = df.iloc[indices] # using only 10,000 rows for PoC to make training quicker

# Retrieval Component

*   Find the most relevant cluster based on the user review
*   Load only the most relevant cluster as the document base for retrieval
*   Use a retriever to find the top k relevant reviews





In [ ]:
def get_topic(user_review):
  return 0

user_review = "Great Italian restaurant with excellent pasta"
topic = get_topic(user_review)

# with open(f'clustering/clusters/topic{topic}.cluster', 'rb') as file:
#     vocab_list = pickle.load(file)

#df = pd.DataFrame([vocab_list])
loader = DataFrameLoader(df, page_content_column = "text")

In [ ]:
# load documents
docs = loader.load()
# split text
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents = splits, embedding = embeddings)
retriever = vectorstore.as_retriever(search_type = 'similarity')

In [ ]:
# HuggingFace pipline
llm_pipeline = pipeline("text-generation", model = "distilgpt2")

In [ ]:
prompt = "Great Italian restaurant with excellent pasta"
retrieved_docs = retriever.get_relevant_documents(prompt)
user_review = prompt

<ipython-input-10-cba8edbc44d0>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(prompt)


In [ ]:
# Display retrieved results
context = ''
print("Retrieved reviews:")
for i, doc in enumerate(retrieved_docs[:5]):
    print(f"Review {i+1}:\n{doc.page_content}\n")
    context += f"Review {i+1}:\n{doc.page_content}\n"

Retrieved reviews:
Review 1:
Local, simple and delicious. This Italian restaurant is economical and family friendly. I had spaghetti and meatballs. Yum

Review 2:
Disappointed in our two pasta meals that were $20 each. The portions were pretty small and the food was barley luke-warm! The didn't offer grated Parmesan cheese or black pepper, which is pretty standard for most Italian restaurants. For dessert we ordered the bread pudding which tasted like it was frozen and heated in a microwave. Also, our waiter who was very nice, had pretty long, greasy hair (sorry) that was not pulled back in any way. For restaurant presentation and health reasons, that should probably be addressed. \nAll in all, I wouldn't recommend this place if you're looking for a delicious Italian meal. You can definitely find other restaurants in the area that are cheaper and tastier!

Review 3:
Excellent food and friendly service in this unpretentious Italian restaurant.\n\n\n I had a delicious melanzane al parmig

In [ ]:
response_prompt = (
    "Based on the following reviews, generate a concise recommendation for a great Italian restaurant with excellent pasta. "
    "The recommendation should mention one restaurant name, the highlights, and why it stands out:\n\n"
    f"{retrieved_docs}\n\n"  # Include retrieved reviews for context
    "Please generate a clear and friendly recommendation."
)

In [ ]:
generated_response = llm_pipeline(response_prompt, max_new_tokens=50, num_return_sequences=1)
print("Generated recommendation:")
print('Generated response: ', generated_response[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated recommendation:
Generated response:  Based on the following reviews, generate a concise recommendation for a great Italian restaurant with excellent pasta. The recommendation should mention one restaurant name, the highlights, and why it stands out:

[Document(metadata={'label': 1}, page_content='Consequently, one MUST sit in the dining area for the corresponding restaurant. \\n\\nNow onto the food.  \\n\\nThe wine list is extensive but only includes Italian selections.  Most of the servers are knowledgeable and can recommend pairings. \\n\\nThe appetizers are decent but are not worth the ~$10-$15 cost.  \\n\\nI\'ve sampled both the pastas and the pizzas.  The pastas are underwhelming in terms of originality, taste, and size. Entrees come with bread and oil for dipping, but the bread is too salty for my enjoyment.  I will never understand why carb-loaded restaurants serve bread with their offerings.  It seems redundant. \\n\\nPizzas are the restaurant\'s specialty (at a place

In [ ]:
print((generated_response)[0])

{'generated_text': 'Based on the following reviews, generate a concise recommendation for a great Italian restaurant with excellent pasta. The recommendation should mention one restaurant name, the highlights, and why it stands out:\n\n[Document(metadata={\'label\': 1}, page_content=\'Consequently, one MUST sit in the dining area for the corresponding restaurant. \\\\n\\\\nNow onto the food.  \\\\n\\\\nThe wine list is extensive but only includes Italian selections.  Most of the servers are knowledgeable and can recommend pairings. \\\\n\\\\nThe appetizers are decent but are not worth the ~$10-$15 cost.  \\\\n\\\\nI\\\'ve sampled both the pastas and the pizzas.  The pastas are underwhelming in terms of originality, taste, and size. Entrees come with bread and oil for dipping, but the bread is too salty for my enjoyment.  I will never understand why carb-loaded restaurants serve bread with their offerings.  It seems redundant. \\\\n\\\\nPizzas are the restaurant\\\'s specialty (at a pla

# Generation

Look into ChatPrompTemplate to make a custom prompt https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html

Models:
Google Gemma [google/gemma-2-2b-it](https://huggingface.co/google/gemma-2-2b-it)

In [ ]:


def get_template(user_review, context):

  YELP_PROMPT_TEMPLATE = f"""The following are customer reviews of a business:

  BEGIN REVIEWS

  {context}

  END REVIEWS

  Based on the above reviews, provide a personalized recommendation to a user who wrote this review: "{user_review}".

  Follow the guidelines below when answering the question:
  1. If something is mentioned in multiple reviews, it is important and more likely to be preferred in the answer.
  2. If the reviews are of a restaurant, bar, or any food establishment, use specific names of dishes, drinks, and desserts.
  3. If the user mentions something that is mentioned in the above reviews, address it in your recommendation.
  """


  return (YELP_PROMPT_TEMPLATE)


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `fafasdasda` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `fafasda

In [ ]:
# from langchain import HuggingFaceHub
# from langchain import PromptTemplate, LLMChain


# prompt = PromptTemplate(template=YELP_PROMPT_TEMPLATE, input_variables=["context", "user_review"])

# os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass()
# llm_chain = LLMChain(prompt=prompt,
#                      llm=HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":1e-10, "max_length": 100}))

# question = "Great Italian restaurant with excellent pasta"
# response = llm_chain.run(user_review=user_review, context=context)
# print(response)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b-it",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device=device,
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# **Testing End-to-End Conversational Review Generation**

User review --> Find similar reviews (context) --> Fill in template with review, context --> Pass to model

In [ ]:
def get_prompt(user_review):
  retrieved_docs = retriever.get_relevant_documents(user_review)
  context = ''
  for i, doc in enumerate(retrieved_docs[:5]):
      context += f"Review {i+1}:\n{doc.page_content}\n"
  return get_template(user_review, context)

good_review = 'Great Italian restaurant with excellent pasta'
# bad_review = "I hate Italian food. I hate pizza. Cheese stinks. Pasta is disgusting. The service at the Italian restaurant sucked. I wish I never have to eat Italian food in my life again."
messages = [
    {"role": "user", "content": get_prompt(good_review)},
]

outputs = pipe(messages, max_new_tokens=256)

assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
print(assistant_response)


Based on your review and the other reviews, you're clearly a fan of great Italian pasta!  Here's what I'd recommend:

**You'll love the [Restaurant Name]'s [Dish Name]**.  It's a classic for a reason, and it sounds like you're already a fan of their pasta.  

**Here's why you'll love it:**

* **The food is consistently praised:**  Multiple reviewers mention the deliciousness of the pasta, and you're already a fan.
* **The atmosphere is cozy and inviting:**  Reviewers mention the restaurant's atmosphere as a big plus, and you've already commented on how much you enjoy the ambiance.
* **The lunch menu is a great value:**  Reviewers rave about the lunch menu, and it sounds like you're looking for a good deal.

**Don't forget to try their [Dish Name]**:  It's a popular choice and a great way to experience the restaurant's signature dishes. 


Let me know if you have any other questions!


### Experimenting with Different Templates

In [ ]:
def get_template(user_review, context):
    YELP_PROMPT_TEMPLATE = f"""The following are customer reviews of a business:

BEGIN REVIEWS

{context}

END REVIEWS

A user wrote this review: "{user_review}".

Analyze the user's review and the provided context. Then, respond based on the following guidelines:

1. Determine if the user is reviewing a specific business or seeking a recommendation. Do not make this explicit in the output. Respond directly to the user.

2. If the user is reviewing a specific business:
   a. Acknowledge the user's positive experiences.
   b. Highlight 2-3 key aspects of the business that align with the user's comments and are supported by other reviews.
   c. Suggest 1-2 other features or services of the business that the user might enjoy based on their preferences.

3. If the user is seeking a recommendation:
   a. Start with a friendly greeting that acknowledges the user's specific interests or preferences.
   b. Recommend 2-3 businesses, items, or features similar to the reviewed business, strictly based on explicit details in the reviews.
   c. For each recommendation, provide a brief explanation of why it's suitable, relating it directly to user comments and review context.

4. In both cases:
   a. Use phrasing like "users frequently mention..." or "customers highlight..." rather than referring to specific review numbers.
   b. Ensure all mentioned features or recommendations are directly supported by the reviews provided or the user's comments.
   c. Do not fabricate details, locations, or links. Only provide details explicitly available in the context.

5. Conclude with a friendly, encouraging statement.

Format your response as a cohesive recommendation or analysis, avoiding explicit references to review numbers in the narrative.

"""

    return YELP_PROMPT_TEMPLATE

In [ ]:
pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b-it",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device=device,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from IPython.display import display, Markdown, Latex

In [ ]:
def get_prompt(user_review):
  retrieved_docs = retriever.get_relevant_documents(user_review)
  context = ''
  for i, doc in enumerate(retrieved_docs[:20]):
      context += f"Review {i+1}:\n{doc.page_content}\n"
  return get_template(user_review, context)

good_review = 'I really enjoyed eating at Noodle Head in Pittsburgh. The Pad Thai was amazing. My only issue was the long wait time. Can you find me similar restaurants?'
# bad_review = "I hate Italian food. I hate pizza. Cheese stinks. Pasta is disgusting. The service at the Italian restaurant sucked. I wish I never have to eat Italian food in my life again."
messages = [
    {"role": "user", "content": get_prompt(good_review)},
]

outputs = pipe(messages, max_new_tokens=256)

assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
display(Markdown(assistant_response))


It sounds like you really enjoyed the Pad Thai at Noodle Head!  Users frequently mention the deliciousness of the Pad Thai and the flavorful broth.  It's a popular choice, and many customers highlight the quality of the noodles.  If you're looking for similar experiences, you might also enjoy exploring other Thai restaurants in the area.  For example, if you're looking for a place with a similar level of flavor and quality, you might try [Name of a Thai restaurant in Pittsburgh] which is known for its authentic dishes.  Another option is [Name of a Thai restaurant in Pittsburgh] which is highly recommended for its generous portions and friendly service.

In [ ]:
df['text'][0]

"dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."